In [20]:
from dhanhq import dhanhq, marketfeed
import talib
import numpy as np
import pandas as pd
import time
import datetime
import os
import logging

In [63]:
with open("token.txt", "r") as file:
    # Read the entire contents of the file
    token = file.read()
client_id = "1102249582"
access_token = token
dhan = dhanhq(client_id,access_token)

masterlist = pd.read_csv('https://images.dhan.co/api-data/api-scrip-master.csv', nrows=1)
masterlist = pd.read_csv('https://images.dhan.co/api-data/api-scrip-master.csv', usecols=masterlist.columns, low_memory=False)


logging.basicConfig(filename="logfile.txt", 
                    level=logging.INFO, 
                    format="%(asctime)s - %(levelname)s - %(message)s")


# Common function to place orders
def place_order(security_id, transaction_type, quantity):
    """
    Place an order using dhan API
    
    Args:
        security_id: Security ID to trade
        transaction_type: dhan.BUY or dhan.SELL
        quantity: Quantity to trade
    """
    print("Inside Place Order", security_id, transaction_type, quantity)
    dhan.place_order(
        security_id=security_id,
        exchange_segment=dhan.NSE_FNO,
        transaction_type=transaction_type,
        quantity=quantity,
        order_type=dhan.MARKET,
        product_type=dhan.INTRA,
        price=0
    )
    pass  # Placeholder since order is commented out


In [64]:
NIFTY_PREMIMUM = 26100

filtered_ce_masterlist = masterlist[
    (masterlist.SEM_INSTRUMENT_NAME == 'OPTIDX') &
    (masterlist.SEM_TRADING_SYMBOL.str.startswith('NIFTY')) &
    (masterlist.SEM_EXPIRY_DATE == '2025-11-04 14:30:00') &
    (masterlist.SEM_STRIKE_PRICE == NIFTY_PREMIMUM) &
    (masterlist.SEM_OPTION_TYPE == 'CE') 
]
filtered_ce_masterlist

,SEM_EXM_EXCH_ID,SEM_SEGMENT,SEM_SMST_SECURITY_ID,SEM_INSTRUMENT_NAME,SEM_EXPIRY_CODE,SEM_TRADING_SYMBOL,SEM_LOT_UNITS,SEM_CUSTOM_SYMBOL,SEM_EXPIRY_DATE,SEM_STRIKE_PRICE,SEM_OPTION_TYPE,SEM_TICK_SIZE,SEM_EXPIRY_FLAG,SEM_EXCH_INSTRUMENT_TYPE,SEM_SERIES,SM_SYMBOL_NAME
153947,NSE,D,47694,OPTIDX,0,NIFTY-Nov2025-26100-CE,75.0,NIFTY 04 NOV 26100 CALL,2025-11-04 14:30:00,26100.0,CE,5.0,W,OP,NaN,NaN


In [24]:
filtered_pe_masterlist = masterlist[
    (masterlist.SEM_INSTRUMENT_NAME == 'OPTIDX') &
    (masterlist.SEM_TRADING_SYMBOL.str.startswith('NIFTY')) &
    (masterlist.SEM_EXPIRY_DATE == '2025-11-04 14:30:00') &
    (masterlist.SEM_STRIKE_PRICE == NIFTY_PREMIMUM) &
    (masterlist.SEM_OPTION_TYPE == 'PE')
]

filtered_pe_masterlist

,SEM_EXM_EXCH_ID,SEM_SEGMENT,SEM_SMST_SECURITY_ID,SEM_INSTRUMENT_NAME,SEM_EXPIRY_CODE,SEM_TRADING_SYMBOL,SEM_LOT_UNITS,SEM_CUSTOM_SYMBOL,SEM_EXPIRY_DATE,SEM_STRIKE_PRICE,SEM_OPTION_TYPE,SEM_TICK_SIZE,SEM_EXPIRY_FLAG,SEM_EXCH_INSTRUMENT_TYPE,SEM_SERIES,SM_SYMBOL_NAME
153946,NSE,D,47693,OPTIDX,0,NIFTY-Nov2025-26050-PE,75.0,NIFTY 04 NOV 26050 PUT,2025-11-04 14:30:00,26050.0,PE,5.0,W,OP,NaN,NaN


In [ ]:

SECURITY_ID = 47694

IS_ACTIVE = False
IS_OPEN_POSITION = False
quantity = 0
buy_price = 0     
stop_loss = 0

current_time = datetime.datetime.now()
current_second = current_time.second
time_str = current_time.strftime("%H:%M:%S")

current_date = datetime.datetime.now().strftime("%Y-%m-%d")

while IS_ACTIVE is False:
        current_second = datetime.datetime.now().second
        time.sleep(1)
        print("waiting to activate", current_second)
        if current_second == 0:
            IS_ACTIVE = True

while IS_ACTIVE is True and IS_OPEN_POSITION is False:
      current_second = datetime.datetime.now().second
      if current_second == 0:
            break
      print('Active',current_second)
      time.sleep(0.15)
      intraday = dhan.intraday_minute_data(str(SECURITY_ID), dhan.NSE_FNO, "OPTIDX", current_date, current_date, 1)

      print("intraday", intraday)
      intraday_df = pd.DataFrame(intraday['data']).tail(2)
      temp_list = []

    

      previous_closed_candle = intraday_df.iloc[-2]
      current_running_candle = intraday_df.iloc[-1]
      
      
     
      # if current_running_candle['close'] > previous_closed_candle['high']:
      #       print('Buy')
      #       length_of_previous_candle = round(previous_closed_candle['high'] - previous_closed_candle['low'])
      #       risk                      = 1000
      #       quantity                  = round(round(risk / length_of_previous_candle) / 75) * 75
      #       print('quantity',quantity)
            
      #       if quantity > 200:
      #             quantity = 200

      #       buy_price = int(current_running_candle['close'])
      #       stop_loss = int(previous_closed_candle['low'])

      #       place_order(SECURITY_ID, dhan.BUY, quantity)

      #       logging.info(f"BUY at {buy_price}")

      #       IS_OPEN_POSITION = True
      #       break;
      print('Buy')
      length_of_previous_candle = round(previous_closed_candle['high'] - previous_closed_candle['low'])
      risk                      = 1000
      quantity                  = 75 #round(round(risk / length_of_previous_candle) / 75) * 75
      print('quantity',quantity)
      
      if quantity > 200:
            quantity = 200

      buy_price = float(current_running_candle['close'])
      stop_loss = float(previous_closed_candle['low'])

      place_order(SECURITY_ID, dhan.BUY, quantity)

      logging.info(f"BUY at {buy_price}")
      print(f"BUY at {buy_price}")
      print(f"STOP LOSS at {stop_loss}") 

      IS_OPEN_POSITION = True
      break




waiting to activate 26
waiting to activate 27
waiting to activate 28
waiting to activate 29
waiting to activate 30
waiting to activate 31
waiting to activate 32
waiting to activate 33
waiting to activate 34
waiting to activate 35
waiting to activate 36
waiting to activate 37
waiting to activate 38
waiting to activate 39
waiting to activate 40
waiting to activate 41
waiting to activate 42
waiting to activate 43
waiting to activate 44
waiting to activate 45
waiting to activate 46
waiting to activate 47
waiting to activate 48
waiting to activate 49
waiting to activate 50
waiting to activate 51
waiting to activate 52
waiting to activate 53
waiting to activate 54
waiting to activate 55
waiting to activate 56
waiting to activate 57
waiting to activate 58
waiting to activate 59
waiting to activate 0
Active 1
intraday {'status': 'success', 'remarks': '', 'data': {'open': [166.3, 173.0, 173.2, 163.65, 158.0, 155.45, 161.6, 152.4, 152.7, 155.5, 160.95, 163.8, 169.3, 173.85, 174.35, 172.55, 175.9

In [61]:

COST_TO_COST_SL = False
TRAILING_SL = True

print(f"Starting position monitoring...")
print(f"Position Status: IS_OPEN_POSITION = {IS_OPEN_POSITION}, Quantity = {quantity}")
print(f"Stop Loss Settings: COST_TO_COST_SL = {COST_TO_COST_SL}, TRAILING_SL = {TRAILING_SL}")
print(f"Buy Price: {buy_price}, Stop Loss: {stop_loss}")
print("-" * 50)

while IS_OPEN_POSITION is True and quantity > 0:
      time.sleep(0.10)
      
      try:
          # Fetch fresh data for real-time monitoring
          intraday = dhan.intraday_minute_data(str(SECURITY_ID), dhan.NSE_FNO, "OPTIDX", current_date, current_date, 1)
          
          # Check if data is available and valid
          if intraday['status'] != 'success' or not intraday.get('data'):
              print("❌ No data available for monitoring, retrying...")
              continue
              
          # Safely create DataFrame with error handling
          data = intraday['data']
          
          # Check if data is a dictionary with lists (expected format)
          if isinstance(data, dict) and all(isinstance(v, list) for v in data.values()):
              intraday_df = pd.DataFrame(data).tail(2)
          else:
              print(f"❌ Unexpected data format: {type(data)}")
              continue
              
          if len(intraday_df) < 2:
              print("❌ Insufficient data for monitoring, retrying...")
              continue

          previous_closed_candle = intraday_df.iloc[-2]
          current_running_candle = intraday_df.iloc[-1]
          
          current_price = current_running_candle['close']
          previous_low = previous_closed_candle['low']
          
          # Calculate P&L
          price_diff = current_price - buy_price
          pnl = price_diff * quantity
          pnl_percentage = (price_diff / buy_price) * 100
          
          # Add timestamp for monitoring
          current_time = datetime.datetime.now().strftime("%H:%M:%S")
          
          # Format P&L display
          if pnl >= 0:
              pnl_display = f"📈 +₹{pnl:.2f} (+{pnl_percentage:.2f}%)"
          else:
              pnl_display = f"📉 ₹{pnl:.2f} ({pnl_percentage:.2f}%)"
          
          # Show appropriate stop loss monitoring based on settings
          if COST_TO_COST_SL is True and TRAILING_SL is False:
              # Cost-to-Cost SL only
              ctc_sl_level = buy_price + 1
              print(f"⏰ {current_time} | 📊 Current: {current_price:.2f} | 🎯 SL: {stop_loss:.2f} | 💰💰 CTC SL: {ctc_sl_level:.2f} | 💰 Buy: {buy_price:.2f} | {pnl_display}")
          elif TRAILING_SL is True and COST_TO_COST_SL is False:
              # Trailing SL only
              print(f"⏰ {current_time} | 📊 Current: {current_price:.2f} | 🎯 SL: {stop_loss:.2f} | 🔄 Trail SL: {previous_low:.2f} | 💰 Buy: {buy_price:.2f} | {pnl_display}")
          elif COST_TO_COST_SL is True and TRAILING_SL is True:
              # Both CTC and Trailing SL
              ctc_sl_level = buy_price + 1
              print(f"⏰ {current_time} | 📊 Current: {current_price:.2f} | 🎯 SL: {stop_loss:.2f} | 💰💰 CTC SL: {ctc_sl_level:.2f} | 🔄 Trail SL: {previous_low:.2f} | 💰 Buy: {buy_price:.2f} | {pnl_display}")
          else:
              # Regular SL only
              print(f"⏰ {current_time} | 📊 Current: {current_price:.2f} | 🎯 SL: {stop_loss:.2f} | 💰 Buy: {buy_price:.2f} | {pnl_display}")

          # Stop loss
          if current_price < stop_loss and COST_TO_COST_SL is False and TRAILING_SL is False:
                print("🔴 STOP LOSS TRIGGERED!")
                print(f"Current Price ({current_price:.2f}) < Stop Loss ({stop_loss:.2f})")
                place_order(SECURITY_ID, dhan.SELL, quantity)
                print(f"✅ SL hit at {current_price:.2f}")
                print(f"💸 Final P&L: ₹{pnl:.2f} ({pnl_percentage:.2f}%)")
                IS_OPEN_POSITION = False
                break

          # Trailing SL
          if current_price < previous_low and COST_TO_COST_SL is False and TRAILING_SL is True:
                print("🔴 TRAILING STOP LOSS TRIGGERED!")
                print(f"Current Price ({current_price:.2f}) < Previous Low ({previous_low:.2f})")
                place_order(SECURITY_ID, dhan.SELL, quantity)
                print(f"✅ Trailing SL hit at {current_price:.2f}")
                print(f"💸 Final P&L: ₹{pnl:.2f} ({pnl_percentage:.2f}%)")
                IS_OPEN_POSITION = False
                break

          # Shift sl to Cost to Cost
          if current_price < buy_price + 1 and COST_TO_COST_SL is True and TRAILING_SL is False:
                print("🔴 COST TO COST STOP LOSS TRIGGERED!")
                print(f"Current Price ({current_price:.2f}) < Buy Price + 1 ({buy_price + 1:.2f})")
                place_order(SECURITY_ID, dhan.SELL, quantity)
                print(f"✅ CTC SL hit at {current_price:.2f}")
                print(f"💸 Final P&L: ₹{pnl:.2f} ({pnl_percentage:.2f}%)")
                IS_OPEN_POSITION = False
                break
                
      except Exception as e:
          print(f"❌ Error monitoring position: {e}")
          time.sleep(0.1)  # Brief pause on error
          continue

print("🏁 Position monitoring ended!")

            

Starting position monitoring...
Position Status: IS_OPEN_POSITION = True, Quantity = 75
Stop Loss Settings: COST_TO_COST_SL = False, TRAILING_SL = True
Buy Price: 201.15, Stop Loss: 199.8
--------------------------------------------------
⏰ 12:39:04 | 📊 Current: 202.30 | 🎯 SL: 199.80 | 🔄 Trail SL: 199.80 | 💰 Buy: 201.15 | 📈 +₹86.25 (+0.57%)
⏰ 12:39:04 | 📊 Current: 202.30 | 🎯 SL: 199.80 | 🔄 Trail SL: 199.80 | 💰 Buy: 201.15 | 📈 +₹86.25 (+0.57%)
⏰ 12:39:05 | 📊 Current: 202.30 | 🎯 SL: 199.80 | 🔄 Trail SL: 199.80 | 💰 Buy: 201.15 | 📈 +₹86.25 (+0.57%)
⏰ 12:39:05 | 📊 Current: 202.30 | 🎯 SL: 199.80 | 🔄 Trail SL: 199.80 | 💰 Buy: 201.15 | 📈 +₹86.25 (+0.57%)
⏰ 12:39:05 | 📊 Current: 202.30 | 🎯 SL: 199.80 | 🔄 Trail SL: 199.80 | 💰 Buy: 201.15 | 📈 +₹86.25 (+0.57%)
❌ No data available for monitoring, retrying...
❌ No data available for monitoring, retrying...
❌ No data available for monitoring, retrying...
⏰ 12:39:05 | 📊 Current: 202.30 | 🎯 SL: 199.80 | 🔄 Trail SL: 199.80 | 💰 Buy: 201.15 | 📈 +₹86.25 (+